In [1]:
!wget -q https://dlcdn.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz
!tar xf spark-3.3.1-bin-hadoop3.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.1-bin-hadoop3"

import findspark
findspark.init()


from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("my_first_pyspark")\
        .config('spark.ui.port','4050')\
        .getOrCreate()

sc = spark.sparkContext

In [2]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
# load할 때 schema를 정의해서 불러오기 

Myschema = StructType([
  StructField("uid", LongType(), False),
  StructField("mid", LongType(), False),
  StructField("rate",LongType(), False),
  StructField("timestamp", LongType(), False)
])

udata = spark.read.format("csv").option("sep", "\t").schema(Myschema).load("/content/drive/MyDrive/Colab Notebooks/데이터분석을위한프로그래밍/u.data")
udata.show(3)

+---+---+----+---------+
|uid|mid|rate|timestamp|
+---+---+----+---------+
|196|242|   3|881250949|
|186|302|   3|891717742|
| 22|377|   1|878887116|
+---+---+----+---------+
only showing top 3 rows



In [3]:
uitemschema = StructType([
  StructField("mid", LongType(), False),
  StructField("title",StringType(), False)
])

uitem = spark.read.format("csv").option("sep", "|").schema(uitemschema).load("/content/drive/MyDrive/Colab Notebooks/데이터분석을위한프로그래밍/u.item")
uitem.show(3)

+---+-----------------+
|mid|            title|
+---+-----------------+
|  1| Toy Story (1995)|
|  2| GoldenEye (1995)|
|  3|Four Rooms (1995)|
+---+-----------------+
only showing top 3 rows



# QUIZ1

In [24]:
udata5 = udata.filter(col("rate")==5)
joinExpression = udata5["mid"]==uitem["mid"]

joinType = "inner"
joined_data = udata5.join(uitem, joinExpression,joinType)

Answer =joined_data.filter(substring(joined_data["title"], 0, 1)=='T').select("title").distinct().count()
print("답은", Answer,"입니다")

답은 75 입니다


# QUIZ2

In [ ]:
udata1 = udata.filter(col("rate")==1)
udata5 = udata.filter(col("rate")==5)
udata5=udata5.selectExpr("mid AS xxx")

joinExpression = udata5["xxx"]==udata1["mid"]
joinType = "inner"
joined_1_5 = udata5.join(udata1, joinExpression,joinType).select("mid").distinct()

joinExpression = joined_1_5["mid"]==uitem["mid"]
joinType = "inner"
joined_asdfas = joined_1_5.join(uitem, joinExpression,joinType)
joined_asdfas.show()

joined_asdfas.select(max(length(joined_asdfas["title"]))).show()

+----+----+--------------------+
| mid| mid|               title|
+----+----+--------------------+
|  26|  26|Brothers McMullen...|
|  29|  29|Batman Forever (1...|
| 964| 964|Month by the Lake...|
| 558| 558|Heavenly Creature...|
| 191| 191|      Amadeus (1984)|
| 418| 418|   Cinderella (1950)|
|1010|1010|     Basquiat (1996)|
|  65|  65|What's Eating Gil...|
|1258|1258|Trial and Error (...|
|1277|1277|   Set It Off (1996)|
| 541| 541|Mortal Kombat (1995)|
| 222| 222|Star Trek: First ...|
| 293| 293|Donnie Brasco (1997)|
| 270| 270|      Gattaca (1997)|
| 938| 938|Smile Like Yours,...|
| 730| 730|Queen Margot (Rei...|
|1127|1127|Truman Show, The ...|
| 705| 705|Singin' in the Ra...|
| 278| 278| Bed of Roses (1996)|
|1463|1463|    Boys, Les (1997)|
+----+----+--------------------+
only showing top 20 rows

+------------------+
|max(length(title))|
+------------------+
|                81|
+------------------+

